# This Notebook is for training new Sato models with other datacorpus

In [ ]:
# enviroment set-up
import os
import sys
sys.path.append("..")
sys.path.append("D:\\20120321_DHBW_AZUREML\\sato")
sys.path.append("D:\\20120321_DHBW_AZUREML\\sato\\model")
from os.path import join

# set env-var
os.environ['BASEPATH'] = 'D:\\20120321_DHBW_AZUREML\\sato'
os.environ['RAW_DIR'] = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/dhbwmlc-ds12-v2/code/Users/svenlangenecker/viznet-master/raw' # path to the raw data
os.environ['SHERLOCKPATH'] = os.environ['BASEPATH']+'\\sherlock'
os.environ['EXTRACTPATH'] = os.environ['BASEPATH']+'\\extract'
#os.environ['PYTHONPATH'] = os.environ['PYTHONPATH']+':'+os.environ['SHERLOCKPATH']
#os.environ['PYTHONPATH'] = os.environ['PYTHONPATH']+':'+os.environ['BASEPATH']
os.environ['TYPENAME'] = 'type_gittables_abstraction'

# set requirements
#from azureml.core import Workspace, Environment
#ws = Workspace.from_config()
#Environment(name='satoEnv')

#satoEnv = Environment.from_pip_requirements(name="satoEnv",file_path="../requirements.txt")
#satoEnv.register(workspace=ws)


In [ ]:
# Column feature extraction
%run ../extract/extract_features.py public_bi_benchmark -f sherlock -n 4

In [ ]:
# split train test
%run ../extract/split_train_test.py --corpus_list gittables-abstraction-tables

In [ ]:
# retrain Sherlock with turl output-size-layer
%run ../model/train_sherlock.py -c ../model/params/turl/sherlock_retrain.txt 

# 1. Sato-Baseline

In [ ]:
# validation of Sato, first try in column-level split
%run ../model/train_CRF_LC.py -c ../model/params/public_bi/CRF+LDA_eval.txt --model_list "model.pt" "CRF+LDA_pathL.pt" "CRF+LDA_pre.pt" --train_percent "train100" --test_percent "test100" --comment "sato_baseline_column_level_split_test20" --column_level_split_file_path "D:\\semantic_data_lake\\semantic_data_lake\\data\\extract\\out\\labeled_unlabeled_test_split\\public_bi_1_absolute_20.0.json"
#%run ../model/train_CRF_LC.py -c ../model/params/publicbi/CRF+LDA_eval.txt --model_list "CRF_pre.pt" "CRF+LDA_retrain_train80_test20.pt" --train_percent "train100" --test_percent "test100" --comment "column-level-split"


In [ ]:
# validation of Sherlock, first try in column-level split
%run ../model/train_sherlock.py -c ../model/params/public_bi/sherlock_eval.txt --model_list "all_None.pt" --comment "sherlock_baseline_column_level_split_test20" --column_level_split_file_path "D:\\semantic_data_lake\\semantic_data_lake\\data\\extract\\out\\labeled_unlabeled_test_split\\public_bi_1_absolute_20.0.json"
#%run ../model/train_sherlock.py -c ../model/params/publicbi/sherlock_eval.txt --model_list "all_None.pt" --comment "sherlock_baseline_column_level_split_test20"

# 2. Sato + small set of labeled training data

In [ ]:
f"Test{+"Test"}"

In [ ]:
import numpy as np

np.concatenate((np.arange(0,6,1),np.arange(6,11,1)))

In [ ]:
import numpy as np
import os

# set env-var
os.environ['BASEPATH'] = 'D:\\20120321_DHBW_AZUREML\\sato'
os.environ['RAW_DIR'] = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/dhbwmlc-ds12-v2/code/Users/svenlangenecker/viznet-master/raw' # path to the raw data
os.environ['SHERLOCKPATH'] = os.environ['BASEPATH']+'\\sherlock'
os.environ['EXTRACTPATH'] = os.environ['BASEPATH']+'\\extract'
#os.environ['PYTHONPATH'] = os.environ['PYTHONPATH']+':'+os.environ['SHERLOCKPATH']
#os.environ['PYTHONPATH'] = os.environ['PYTHONPATH']+':'+os.environ['BASEPATH']
os.environ['TYPENAME'] = 'type78'

#for index,percent in enumerate(np.arange(15,55,5)):
for index, percent in enumerate(np.arange(1,2,1)):
    # if index != 0:
    #     continue
    comment = f"labeled{percent}_unlabeledAbsolute_test{20}"
    column_level_split_file_path = f"D:\\semantic_data_lake\\semantic_data_lake\\data\\extract\\out\\labeled_unlabeled_test_split\\public_bi_{percent}_absolute_20.0.json"
    pretrained_shelock_path = f"sherlock_retrain_labeled{percent}_unlabeledAbsolute_test{20.0}.pt"
    pretrained_CRF_LDA_path = f"CRF+LDA_retrain_labeled{percent}_unlabeledAbsolute_test{20.0}.pt"
    
    # retrain sherlock
    %run ../model/train_sherlock.py -c ../model/params/public_bi/sherlock_retrain.txt  --comment {comment} --column_level_split_file_path {column_level_split_file_path}
    # retrain sato
    %run ../model/train_CRF_LC.py -c ../model/params/public_bi/CRF+LDA_retrain.txt --pre_trained_sherlock_path {pretrained_shelock_path} --comment {comment} --column_level_split_file_path {column_level_split_file_path}
    # validate sato
    %run ../model/train_CRF_LC.py -c ../model/params/public_bi/CRF+LDA_eval.txt --model_list {pretrained_CRF_LDA_path} --comment {"eval_"+comment} --column_level_split_file_path {column_level_split_file_path}

# 4. Sato retrained with EmbClus

In [ ]:
import numpy as np
import os

# set env-var
os.environ['BASEPATH'] = 'D:\\20120321_DHBW_AZUREML\\sato'
os.environ['RAW_DIR'] = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/dhbwmlc-ds12-v2/code/Users/svenlangenecker/viznet-master/raw' # path to the raw data
os.environ['SHERLOCKPATH'] = os.environ['BASEPATH']+'\\sherlock'
os.environ['EXTRACTPATH'] = os.environ['BASEPATH']+'\\extract'
#os.environ['PYTHONPATH'] = os.environ['PYTHONPATH']+':'+os.environ['SHERLOCKPATH']
#os.environ['PYTHONPATH'] = os.environ['PYTHONPATH']+':'+os.environ['BASEPATH']
os.environ['TYPENAME'] = 'type78'

distance_threshold = 0.1
current_corpus = "public_bi"

#for index,percent in enumerate(np.arange(5,55,5)):
#for index, percent in enumerate([1,2,3,4,6,7,8,9,10,15,20,25,30,35,40,45,50]):
for index, percent in enumerate([1,2,3,4,5]):
    #for distance_threshold in [1e-10,1e-9,1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e-0]:
    for distance_threshold in [1e-4]:
        # if index > 0:
        #     break
        comment = f"embclus_without_knn_{distance_threshold}_labeled{percent}_unlabeledAbsolute_test{20.0}"
        column_level_split_file_path = f"D:\\semantic_data_lake\\semantic_data_lake\\data\\extract\\out\\labeled_unlabeled_test_split\\{current_corpus}_{percent}_absolute_20.0.json"
        #embclus_gen_train_data_path = f"D:\\semantic_data_lake\\semantic_data_lake\\emb_clus\\knn_classifier\\out\\gen_training_data\\public_bi_gen_training_data_1_{distance_threshold}_{percent}_absolute_20.0.csv"
        embclus_gen_train_data_path = f"D:\\semantic_data_lake\\semantic_data_lake\\emb_clus\\without_knn\\out\\gen_training_data\\{current_corpus}_gen_training_data_{distance_threshold}_{percent}_absolute_20.0.csv"
        pretrained_shelock_path = f"sherlock_retrain_embclus_without_knn_{distance_threshold}_labeled{percent}_unlabeledAbsolute_test{20.0}.pt"
        pretrained_CRF_LDA_path = f"CRF+LDA_retrain_embclus_without_knn_{distance_threshold}_labeled{percent}_unlabeledAbsolute_test{20.0}.pt"
        
        # retrain sherlock
        %run ../model/train_sherlock.py -c ../model/params/public_bi/sherlock_retrain.txt --comment {comment} --column_level_split_file_path {column_level_split_file_path} --embclus_gen_train_data_path {embclus_gen_train_data_path}
        #%run ../model/train_sherlock.py -c ../model/params/public_bi/sherlock_retrain.txt --comment {comment} --column_level_split_file_path {column_level_split_file_path}

        # retrain sato
        %run ../model/train_CRF_LC.py -c ../model/params/public_bi/CRF+LDA_retrain.txt --pre_trained_sherlock_path {pretrained_shelock_path} --comment {comment} --column_level_split_file_path {column_level_split_file_path} --embclus_gen_train_data_path {embclus_gen_train_data_path}
        #%run ../model/train_CRF_LC.py -c ../model/params/public_bi/CRF+LDA_retrain.txt --pre_trained_sherlock_path {pretrained_shelock_path} --comment {comment} --column_level_split_file_path {column_level_split_file_path}
        # validate sato
        %run ../model/train_CRF_LC.py -c ../model/params/public_bi/CRF+LDA_eval.txt --model_list {pretrained_CRF_LDA_path} --comment {"eval_"+comment} --column_level_split_file_path {column_level_split_file_path}

In [2]:
%run run_exp_4_without_knn_combinedLfs.py

KeyboardInterrupt: 